<a href="https://colab.research.google.com/github/shiehn/dawnet-remotes/blob/main/DAWNet_Remote_MusicGen.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

GPU CHECK

In [ ]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

INSTALL MUSIC_GEN

In [ ]:
!python3 -m pip install -U git+https://github.com/facebookresearch/audiocraft#egg=audiocraft
# !python3 -m pip install -U audiocraft

from audiocraft.models import musicgen
from audiocraft.utils.notebook import display_audio
import torch

model = musicgen.MusicGen.get_pretrained('medium', device='cuda')



In [4]:
import locale
locale.getpreferredencoding = lambda: "UTF-8"

!pip install dawnet-client


import os
import soundfile as sf
import dawnet_client.core as dawnet
from dawnet_client import DAWNetFilePath

placeholder_txt = "Enter the token generated by the DAWNet plugin"
DAWNET_TOKEN = "2baca3a5-f8e4-4c21-bd5e-b09e1fd75481" #@param {type:"string"}
dawnet_token = DAWNET_TOKEN

if dawnet_token is None or dawnet_token == "" or dawnet_token == placeholder_txt:
  print("ERROR: The token provided is not valid.")
  exit()

async def dawnet_func(audio_description: str = "hiphop",
                      audio_length: int = 8,
                      num_samples: int = 1):
    try:
        model.set_generation_params(duration=audio_length)

        for i in range(num_samples):
          output = model.generate(
              descriptions=[audio_description],
              progress=True, return_tokens=True
          )

          output_audio_name = f"generated_audio_{i}.wav"

          # Move the tensor to CPU, convert to numpy array, reshape and ensure the data type is float32
          generated_audio = output[0].cpu().numpy().astype('float32')
          generated_audio = generated_audio.squeeze()  # Remove extra dimensions
          sf.write(output_audio_name, generated_audio, 32000, subtype='PCM_16')  # Save as WAV file


          # after executing your custom code you send data back to the plugin like so ..
          await dawnet.results().add_file(output_audio_name, "wav")


        await dawnet.results().add_message("Returning the input file")
        await dawnet.results().send()

        return True
    except Exception as e:
        print(f"Error in arbitrary_method: {e}")
        return f"Method encountered an error: {e}"


dawnet.set_token(token=dawnet_token)
dawnet.set_name("MusicGen Text-To-Audio")
dawnet.set_description("Provide a text description, number of samples, and sample lenth in seconds")
dawnet.register_method("dawnet_func", dawnet_func)

dawnet.connect_to_server()

WHATS_THE_MESSAGE: 71368a5e-295c-4a9a-a4c0-a92b477d038e


KeyboardInterrupt: ignored

See our repo https://github.com/facebookresearch/audiocraft for more details on how to use this model!

See also [MusicGen Gradio Demo](https://colab.research.google.com/drive/1-Xe9NCdIs2sCUbiSmwHXozK6AAhMm7_i?usp=sharing) for a Colab using the Gradio app instead!
